<a href="https://colab.research.google.com/github/soraochi1626-cmd/economics-study/blob/main/econometrics/multiple_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install py4macro
!pip install wooldridge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 13.1 MB/s eta 0:00:00


In [7]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
import py4macro
import wooldridge

from numba import njit
from pandas.plotting import scatter_matrix
from scipy.stats import norm, uniform, gaussian_kde, multivariate_normal
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif

# 警告メッセージを非表示
import warnings
warnings.filterwarnings("ignore")

#説明
説明変数が複数の重回帰分析(Multiple Regression)

$$y_i = \beta_0 + \sum_{j=1}^{k}{\beta_j}{x_{ij}} + u_i$$

- $i = 1, 2, …, n$：観測地のインデックス
- $i = 1, 2, …, k$：説明変数の数
(注意)

$x_1$と書く場合, $x_{i1}, x_{i1},\dots, x_{n1}$を表す(第一の説明変数)。

一般的に$x_j$と書く場合, $x_{ij}, x_{ij},\dots, x_{nj}$を表す(第j番目の説明変数)。





#statsmodelsを使う

##推定
単回帰分析を使った同じデータセットwage1を使い, 賃金と教育の関係を再考する。前章と異なる点は, 複数の説明変数を導入し重回帰分析を行うことである。追加的な説明変数として次の変数を加える。
- exper：潜在的な経験年数(=年齢ー再考学歴までの年数−６)
- tenure：現在の職場での在職年数
statsmodelsを使う上で単価域となる点は, それぞれの説明変数の前に+を使いするだけである。

まず必要な変数だけを取り出し変数wage1に割り当てる。

In [8]:
wage1 = wooldridge.data('wage1').loc[:,['wage', 'educ', 'tenure', 'exper']]

In [9]:
wooldridge.data('wage1',description=True)

name of dataset: wage1
no of variables: 24
no of observations: 526

+----------+---------------------------------+
| variable | label                           |
+----------+---------------------------------+
| wage     | average hourly earnings         |
| educ     | years of education              |
| exper    | years potential experience      |
| tenure   | years with current employer     |
| nonwhite | =1 if nonwhite                  |
| female   | =1 if female                    |
| married  | =1 if married                   |
| numdep   | number of dependents            |
| smsa     | =1 if live in SMSA              |
| northcen | =1 if live in north central U.S |
| south    | =1 if live in southern region   |
| west     | =1 if live in western region    |
| construc | =1 if work in construc. indus.  |
| ndurman  | =1 if in nondur. manuf. indus.  |
| trcommpu | =1 if in trans, commun, pub ut  |
| trade    | =1 if in wholesale or retail    |
| services | =1 if in services indus.  

被説明変数に対数賃金を使うために, 新たなwage_logを追加する。

In [10]:
wage1['wage_lig'] = np.log(wage1['wage'])

回帰式では追加の説明変数の前に+をつける。

In [11]:
formula_1 = 'wage_lig ~ educ + tenure + exper'

すいて結果を変数res_1に割り当てる

In [12]:
res_1 = smf.ols(formula_1, data=wage1).fit()

次のコードで結果を表にまとめて表示することができる。

In [13]:
print(res_1.summary())

                            OLS Regression Results                            
Dep. Variable:               wage_lig   R-squared:                       0.316
Model:                            OLS   Adj. R-squared:                  0.312
Method:                 Least Squares   F-statistic:                     80.39
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           9.13e-43
Time:                        13:23:58   Log-Likelihood:                -313.55
No. Observations:                 526   AIC:                             635.1
Df Residuals:                     522   BIC:                             652.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2844      0.104      2.729      0.0

#属性とメソッド
推定結果res_1の属性とメソッドdir()やpy4macro.see()で確認できる。以下ではpy4macro.see()を使う。

In [14]:
py4macro.see(res_1)

.HC0_se                   .HC1_se                   .HC2_se                   .HC3_se
.aic                      .bic                      .bse                      .centered_tss
.compare_f_test()         .compare_lm_test()        .compare_lr_test()        .condition_number
.conf_int()               .conf_int_el()            .cov_HC0                  .cov_HC1
.cov_HC2                  .cov_HC3                  .cov_kwds                 .cov_params()
.cov_type                 .df_model                 .df_resid                 .diagn
.eigenvals                .el_test()                .ess                      .f_pvalue
.f_test()                 .fittedvalues             .fvalue                   .get_influence()
.get_prediction()         .get_robustcov_results()  .info_criteria()          .initialize()
.k_constant               .llf                      .load()                   .model
.mse_model                .mse_resid                .mse_total                .nobs
.normalized_cov_pa

単回帰分析と同様に, 係数の推定は属性.paramsで取得できる。

In [15]:
res_1.params

,0
Intercept,0.284360
educ,0.092029
tenure,0.022067
exper,0.004121


res_1はSeriesなので, インデックスを使ってeducの係数だけを取り出す場合は次のようにする。

In [16]:
res_1.params.iloc[1]

np.float64(0.09202898676928341)

次に, メソッドpredict()を使って予測値を計算してみる。次の例では, educ, tenure, experの平均値でのnp.log(wage)を計算する。

In [17]:
wm = wage1.mean() #変数の平均の計算
wm

,0
wage,5.896103
educ,12.562738
tenure,5.104563
exper,17.017110
wage_lig,1.623268


wmはSeriesなので, 変数の行ラベルを使いwm['educ']でeducの平均値だけを取り出すことができる。

In [18]:
wm['educ']

np.float64(12.562737642585551)

predict()に渡す引数としては, 説明変数の辞書を作るのが一番簡単である。

In [19]:
z = {'educ':wm['educ'],
     'tenure':wm['tenure'],
     'exper':wm['exper']}

In [20]:
res_1.predict(z)

,0
0,1.623268


3つの変数が平均値を取る場合の平均時給の対数値は約1.62である

複数の予測値を計算するには, 辞書の値にlistやarrayを書き込めばいい。次の例では, tenureとexperを平均値に固定し, educを平均値, 平均値-1, 平均値+1に変化させて予測値を計算している。

In [21]:
educ_mean = wm['educ']
tenure_mean = wm['tenure']
exper_mean = wm['exper']

z2 = {'educ':[educ_mean-1, educ_mean, educ_mean+1],
                  'tenure':[tenure_mean]*3,
                  'exper':[exper_mean]*3}

educ_return = res_1.predict(z2)
educ_return

,0
0,1.531239
1,1.623268
2,1.715297


３つの変数が平均の値を取る場合, 平均賃金の対数値は1.623268であり, 上の計算と同じになることが確認できる。教育年数が1年減少すると予測値は1.531239であり, 教育年数が1年増えると予測値は1.715297となる。この結果を使い, 教育年数が一年変化する場合の影響を計算してみる。まず教育年数が1年減少する場合を計算する。

In [22]:
educ_return[0]-educ_return[1]

np.float64(-0.0920289867692834)

次に教育年数が1年増える場合を計算する。

In [23]:
educ_return[2] - educ_return[1]

np.float64(0.0920289867692834)

これらの数字の絶対値は同じである。この結果は偶然ではない。他の要因を固定し教育年数が1年変化（減少もしくは増加）した場合，平均時給の対数値がどれだけ変化したかを計算している。これは正しく教育の収益率である。この点を数式で確認するために，変数
の値が小さい場合（例えば，0.02），次の近似が成立することを思い出そう。
$$log(x + 1) \approx x \tag{1}$$
また教育年数が$h$の場合の平均時給を*w*(h)としよう。教育年数が一年増えた場合の教育の収益率は$\frac{w(h + 1) - w(h)}{w(h)}$となり(1)を使うと次のように書くことができる。

$$\begin{align}
\frac{w(h + 1) - w(h)}{w(h)} &\approx
log(\frac{w(h + 1) - w(h)}{w(h)} + 1) \\
&=log(\frac{w(h + 1)}{w(h)})  \tag{2}\\
&=log(w(h + 1)) - log(w(h))
\end{align}$$

最後の等号が上の計算で使った式である。h + 1をh - 1に変えると教育年数が1年減少した場合を考えることができる。

もう一点付け加えるために, 上の回帰分析の変数educの係数の解釈を思い出そう。教区年数が一年増えた場合, 平均賃金の対数値はどれだけ変化するかを示しており, 教育の収益率を表している。従って, ここで計算した値を同じになるはずである。


In [24]:
res_1.params['educ']

np.float64(0.09202898676928341)

predict()の引数を省略すると予測値を返す属性.fittedvaluesと同じ値を返す。それを確かめるために次のコードを評価してみる。

In [25]:
( res_1.predict() == res_1.fittedvalues ).all()

np.True_

Trueが返されているので、値は全て同じだと確認できた。

#変数の変換
上の重回帰分析では，回帰式のwageを対数化した変数を新たに作成し計算を行なっているが、回帰式の中で変数の変換を直接指定することもできる。以下で説明する方法は被説明変数・説明変数の両方に使用可能である。

##Numpyをの関数を使う方法
numpyの対数関数を使って, 回帰式の中で直接書き換えることができる。wageを大数化する例を考える。

In [26]:
formula_2 = 'np.log(wage) ~ educ + tenure + exper'

In [27]:
res_2 = smf.ols(formula_2, data=wage1).fit()

In [28]:
res_2.params

,0
Intercept,0.284360
educ,0.092029
tenure,0.022067
exper,0.004121


##$I()$を使う方法
次にexperの二乗を回帰式に入れるケースを考える。この場合, np.square(exper)でも良いが, $I()$の()の中に直接指揮を書くことが可能となる。

In [29]:
formula_3 = 'np.log(wage) ~ educ + tenure + exper + I(exper**2)'
res_3 = smf.ols(formula_3, data=wage1).fit()

In [30]:
res_3.params

,0
Intercept,0.198345
educ,0.085349
tenure,0.020841
exper,0.032854
I(exper ** 2),-0.000661


この方法であれば, Numpyにない関数も直接書くことができる。

##defを使う方法
より複雑な関数であればdefを使い, 関数を定義し, それを回帰式の中で使う方が良いのかもしれない。experの2畳を回帰式に入れるケースを考える。

In [31]:
def myfunc(x):
  return x**2

In [32]:
formula_4 = 'np.log(wage) ~ educ + exper + tenure + myfunc(exper)'
res_4 = smf.ols(formula_4, data=wage1).fit()

In [33]:
res_4.params

,0
Intercept,0.198345
educ,0.085349
exper,0.032854
tenure,0.020841
myfunc(exper),-0.000661


#係数の解釈
次の推定式を考える。
$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + \beta_3x_3 + u$$

**＜解釈１：Ceteris Paribus＞**

$x_1$以外の変数を固定して$x_1$だけを少しだけ変化させる。これにより, $y$に対する$x_1$の影響だけを考えることが可能になり, 次の記号で表す。
$$\frac{Δy}{Δx_1} = \beta_1$$
ここで
- $Δx_1 = x_1$の変化（1単位の変化と解釈する）
- $Δy = y$の変化

$\beta_1$は, $x_1$が1単位増加した場合の$y$の変化を示している。重要な点は, $x_1$以外の変数は固定されていることである。これをCeteris Paribus（ラテン語;"other things being equal"という意味）と呼ぶ。

**＜解釈２：影響の除去＞**

まず$y$と$x_1$の関係を捉える$\hat{\beta_1}$を考える。もし推定式に$x_2$と$x_3$が入っていなければ, $\hat{\beta_1}$は$x_2$と$x_3$の影響を拾うことになる（$x_1$が$x_2$と$x_3$のそれぞれと完全に無相関でない場合）。即ち, $x_2$と$x_3$が入ることにより,$\hat{\beta_1}$はこれらの変数の影響を「除去」した後に残る$y$と$x_1$の関係を捉えていると考えることができる。同じように, $\hat{\beta_j}$は$j$以外の説明変数の影響を取り除いた後に残る$y$と$x_j$の関係を示している。

wage1を使い以下を推定する。

In [41]:
formula_1a = 'wage ~ educ + tenure + exper'
res_1a = smf.ols(formula_1a, data=wage1).fit()
res_1a.params

,0
Intercept,-2.872735
educ,0.598965
tenure,0.169269
exper,0.022340


tenureとexperを固定して，educを一単位増加させるとしよう。その効果は
$$\frac{Δwage}{Δeduc} ≈ 0.599$$
即ち, 教育期間が一単位（ここでは一年）増えると賃金は0.599ドル増加することを示している。

次にformula_1の回帰式を使って推定したres_1を考える。この場合,

$$\frac{Δln(wage)}{Δeduc} ≈ 0.092$$

対数の性質を使い左辺は次のように書き換えることができる。

$$\frac{Δln(wage)}{Δeduc} = \frac{Δwage/wage}{Δeduc} = \frac{％Δwage}{Δeduc}$$

ここで, ％$Δwage$はwageの変化を意味する。従って, 教育期間が一単位（ここでは一年）増えると賃金は約9.2%増加することを示している。


#ガウス・マルコフの定理

##母集団回帰式と標本回帰式
今までは会期式に基づいてPythonを使い毛いい数の推定方法を説明したが, ここではOLS推定の望ましい性質とは何か, 望ましい性質の条件とは何かを考える。そのために, まず母集団回帰式と標本回帰式について説明する。

計量経済学では, データを使い**経済理論（経済学的な考え）**に基づいた仮説を検証する。例として「消費は家系の所得$x_1$が高くなると考え, こどもの数$x_2$とも正比例する」という仮説を考える。所得とお子供の数に依存する消費の部分だけを$y* = f(x_1,x_2)$となる。さらに, 線形を仮定すると時式となる。

$$y* = \beta_0 + \beta_1x_1 + \beta_2x_2 \tag{1}$$

例えば, 所得30万円, 子ども2人の家計の場合, 消費は$\beta_0 + \beta_1(30万円) + \beta_2(2人)$となる。所得$x1$が変化すると消費$y*$も変化することがわかる。このことは$\beta_2 = 0$の単価域の場合（消費が子供の数に依存しない場合）を考えればもっとわかりやすいだろう。その場合, (1)は

$$y* = \beta_0 + \beta_1x_1$$

となり, 横軸に$x_1$, 縦軸に$y*$を置く図では右肩上がりの直線となる。

(1)は**平均**の家計の行動を表していると考えることができる。従って, 同じ所得と子供の数の家計でも好み等の要因が違うと消費に違いが出る。その違いを（平均ゼロ）の誤差項$u$として捉えると, 全ての家計のしょうひ$y$は次式で表すことができる。

$$y = y* + u \tag{2}$$

- $y*$：平均の家計の消費であり, $x_1$と$x_2$のみに依存する部分
- $u$：平均の家計と異なる家計の消費であり, 平均家計との違いを捉える部分

(1)を(2)に代入すると

$$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + u \tag{3}$$

となる。この式は仮説に基づいた関係式であり, **母集団回帰式**と呼ばれる。また(3)を使い$x_1$と$x_2$を所与として期待値を計算すると時式を得る。

$$E(y|x_1,x_2) = y*  \because E(u) = 0$$

これは「$x_1$と$x_2$を所与とする」という条件の下での期待値なので**条件付き期待値**(conditional expectations)と呼ばれる。この式は, $y$の条件付き期待値は母集団回帰式の
$x_1$と$x_2$に依存する部分である$y*$と等しいことを示している。即ち, (1)は母集団で平均として成立する**母集団回帰線**を表している。



母集団回帰式(式３)が成立するという仮定のもとで母集団パラメータ$\beta_j$, $j = 0,1,2$を推定することになる。そのためには,
- ${y_i, x_i1, x_i2}, i = 1,2,…,n$の標本を収集する。
- 次式に重回帰分析の手法を適用しOLS推定値を計算する。

$$y_i = \beta_0 + \beta_1x_{i1} + \beta_2x_{i2} + u_i, i = 1,2,…,n \tag{4}$$

母集団回帰式(3)と区別するために, この式には添え字$i$があり**標本回帰式**と呼ぶ。


一方で, 上で述べたようにOLS推定式にはなんらかの「望ましい性質」がある。
1. 「望ましい性質」の前提となるっ条件は何か？
2. OLS推定量の「望ましい性質」とは何か？
以下では１から考える。

##仮定と望ましい性質

- 仮定１：Linear in Parameters（母集団回帰式の線形性）
  - 母集団のモデルは時式のようにパラメータに関して線形である。
  $$y = \beta_0 + \beta_1x_1 + \beta_2x_2 + … + \beta_kx_k + u$$
  - $y = \beta_0 + \beta_1x_1 + \beta_2x_1^2$は線形性を満たす。
  - $y = \beta_0 + \frac{\beta_1}{1 + \beta_2x_1} + \beta_3x_2$は線形性を満たさない。
- 仮定２：Randam Sampling（標本の無作為抽出）
  - 無作為に$\{y_i, x_{i1}, x_{i2}, …, x_{ik}\}$,$i = 1, 2, …,n$が抽出される。
  - 多くの横断面データでは成立すると考えられている（時系列データでは満たされない場合が多い）。
  - この仮定により
  $$Corr(u_i,u_j|X) = 0, i \neq j,   X = \{x_1, x_2, …, x_k\}$$
  となる。つまり, 説明変数$X$を所与として$u_i$と$u_j$はなんの関係もないという意味である。
- 仮定２：No Perfect Collinearity（母集団回帰式の説明変数の非完全多重共線性）
  - 説明変数$x_j$が他の説明変数の完全な線形結合（例えば, $x_2 = 1 + 2x_2$や$x_1 = x_2 + 5x_4$）として表すことができないという意味である。もしこの仮定が満たされなければ$x_j$はなんの追加定期な情報を持たなくなる。
  - 定数項以外の説明変数が全く変化しないことも排除している（例えば, $x_j = 3$）。このような変数は定数項の単なるスケールアップ・バージョンであり, 追加的な情報はない。（定数項以外の説明変数は変化してこそ$y$を説明できる）
- 仮定４：Zero Conditional Mean（母集団回帰式の誤差項の条件付き期待値は０）
$$E(u|X) = 0, X = \{x_1, x_2, …,x_k\}$$
  - $X$はランダム変数であるが, $X$を所与とすると$u$の平均はゼロになるという仮定である。即ち, ランダム変数である$Z$がどのような値を取ったとしても, $u$の平均は０だということである。$X$は$u$に対してなんの情報も影響力も全く持たないため, $X$と$u$が線形・非線形な関係がないことを意味する。即ち, $X$と$u$は独立性を満たす。
  - $X$はランダム変数であるため, $X$が変化する場合の$E(u|X)$の平均を考えることができる。即ち, $E[E(u|X)] = 0$である。外側の$E$は$X$に対する期待値演算子であり, $E$が入れ子になっているのでiterated expectationsと呼ばれる。$X$がどのような値を取ったとしても仮定４のもとでは０になるので等号が成立することになる。さらに, $X$に対しての期待値が付け加えられているので, $X$を省いて$E[E(u|X)] = E(u) = 0$となる。$X$の条件をつけなくとも$u$の平均はゼロになることを示している。
  - $X$と$u$には線形の依存関係がないので$Cov(X,u) = 0$が成立する。しかし, $Cov(X, u) = 0$（無相関）は$E(u|X) = 0$（独立性）を意味しないことに注意したい。
  


仮定１〜４の下で母集団パラメータのOLS推定量は不偏性（unbiasdness）を満たす。
$$E(\hat{\beta_j}) = \beta_j, j = 0, 1, …, k$$
（解釈）

標本の大きさがnの標本を使い回帰分析をおこない$\hat{\beta_j}$を得たとする。さらに同じ標本の大きさnの異なる標本を使い回帰分析を合計N回$\hat{\beta_j}$を計算したとする。普遍性とはNが十分に大きい場合（$N → ∞$）, N個ある$\hat{\beta_j}$の平均は$\beta_j$と等しいという性質である。

（注意点）

この結果は標本の大きさnに依存しない。即ち, 大標本（$n → ∞$）である必要なない。むしろ小標本（$n < ∞$）で重要視される小標本特性（データが少な場合の特性）と考えるべきである。